<a href="https://colab.research.google.com/github/TechnoPolizzz/safety_doors/blob/main/SafetyDoors_cloud_clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install open3d

In [2]:
import open3d as o3d
import cv2
import numpy as np

In [3]:
from google.colab.patches import cv2_imshow # Позволяет выводить изображения

In [169]:
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

In [5]:
def joinPath(path_name):
  return os.path.join(path_name["path"], path_name["name"])

In [170]:
# Объединение списков без повторений
def mergeLists(list1, list2):
  return list(set(list1 + list2))

### DEPRECATED

In [139]:
def getCubePath(minPt=[0,0,0], maxPt=[1,1,1]):
  # Создаем углы куба
  FLAG_X = 4
  FLAG_Y = 2
  FLAG_Z = 1
  # points = [[0,0,0],[1,0,0],[1,1,0],[0,1,0],[]]
  points = []
  for i in range(8):
    x = maxPt[0] if (i & FLAG_X) else minPt[0]
    y = maxPt[1] if (i & FLAG_Y) else minPt[1]
    z = maxPt[2] if (i & FLAG_Z) else minPt[2]
    points.append([x,y,z])

  # Создаем траекторию обхода этих углов
  path = [0,1,3,2,0,4,5,1,5,7,3,7,6,2,6,4]
  trace = []
  for i in path:
    trace.append(points[i])
  
  return np.asarray(trace)

In [140]:
def getCubePath(points):
  # Создаем траекторию обхода этих углов
  path = [0,1,3,2,0,4,5,1,5,7,3,7,6,2,6,4]
  trace = []
  for i in path:
    trace.append(points[i])
  return np.asarray(trace)

### Разархивируем датасеты

In [ ]:
!unzip /content/drive/MyDrive/technoPolizzz/test_img.zip 

In [ ]:
!unzip /content/drive/MyDrive/technoPolizzz/point_cloud_train.zip

Archive:  /content/drive/MyDrive/technoPolizzz/point_cloud_train.zip
replace point_cloud_train/clouds_stereo/cloud_2_1620665786667079.pcd.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os

# Папка с датасетом
root_dir = "/content/point_cloud_train"

# Список пар: путь к архиву, имя архива
dir_file_arr = []
for file in os.listdir(root_dir):
  sub_dir = os.path.join(root_dir, file)
  for subfile in os.listdir(sub_dir):
    file_dict = {}
    file_dict["path"] = sub_dir
    file_dict["name"] = subfile
    dir_file_arr.append(file_dict)

Получаем имя файла без расширения архива

In [ ]:
zipname = dir_file_arr[0]["name"]
print(zipname)
print(os.path.splitext(zipname)[0])

cloud_0_1620667028472703.pcd.zip
cloud_0_1620667028472703.pcd


In [ ]:
joinPath(dir_file_arr[0])

'/content/point_cloud_train/clouds_tof/cloud_0_1620667028472703.pcd.zip'

Разархивируем файл

In [ ]:
!mv /content/point_cloud_train/clouds_stereo/*.pcd /content/point_cloud_train/clouds_stereo/pcd/

In [ ]:
!mv /content/point_cloud_train/clouds_tof/pcd/* /content/pcd/clouds_tof/

In [ ]:
import zipfile

In [ ]:
for dir_file in dir_file_arr:
  with zipfile.ZipFile(joinPath(dir_file), 'r') as zip_ref:
      zip_ref.extractall(dir_file["path"])
      print("Extracted to ", dir_file["path"])

### Функции для визуализации

In [100]:
# Создаем mesh из обрамляющего прямоугольника
def createMeshFromBB(bb):
  bbcloud = o3d.geometry.PointCloud()
  bbcloud.points = bb.get_box_points()
  bb.color = bb.color
  hull, _= bbcloud.compute_convex_hull()
  return hull

In [125]:
def createMeshData(mesh, opacity=0.2):
  verts = np.asarray(mesh.vertices)
  triangs = np.asarray(mesh.triangles)

  mdata = go.Mesh3d(
        x=verts[:,0],
        y=verts[:,1],
        z=verts[:,2],
        i = triangs[:,0],
        j = triangs[:,1],
        k = triangs[:,2],
        opacity=opacity,
    )
  return mdata

In [133]:
def createScatter3dData(pcd):
  points = np.asarray(pcd.points)
  colors = np.asarray(pcd.colors)
  scdata = go.Scatter3d(
            x=points[:,0], 
            y=points[:,1], 
            z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors))
  return scdata

In [158]:
def drawGeometry(geometry):
  mydata = []
  for g in geometry:
    if type(g) is o3d.geometry.PointCloud:
      mydata.append(createScatter3dData(g))
    elif type(g) is o3d.geometry.AxisAlignedBoundingBox or type(g) is o3d.geometry.OrientedBoundingBox:
      m = createMeshFromBB(g)
      mydata.append(createMeshData(m))
    elif type(g) is o3d.geometry.TriangleMesh:
      mydata.append(createMeshData(g))
  
  fig = go.Figure(
    data=mydata,
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
  )
  fig.show()

In [225]:
def display_inlier_outlier(cloud, ind, in_color=None, out_color=[1, 0, 0]):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    if not out_color is None:
      outlier_cloud.paint_uniform_color(out_color)
    if not in_color is None:
      inlier_cloud.paint_uniform_color(in_color)
    drawGeometry([inlier_cloud, outlier_cloud])

### Работа с облаками

Получаем массивы имен облаков

In [8]:
import os

# Папка с датасетом
root_dir = "/content/pcd"

# Список пар: путь к архиву, имя архива
pcd_files = {}
for file in os.listdir(root_dir):
  pcd_files[file] = []
  sub_dir = os.path.join(root_dir, file)
  for subfile in os.listdir(sub_dir):
    file_dict = {}
    file_dict["path"] = sub_dir
    file_dict["name"] = subfile
    pcd_files[file].append(file_dict)

In [9]:
print(pcd_files.keys())

dict_keys(['.ipynb_checkpoints', 'clouds_tof', 'clouds_stereo'])


In [164]:
index = 15
path_to_stereo = joinPath(pcd_files['clouds_stereo'][index])
path_to_tof = joinPath(pcd_files['clouds_tof'][index])

Открываем облако

In [165]:
stereo_pcd = o3d.io.read_point_cloud(path_to_stereo)
tof_pcd = o3d.io.read_point_cloud(path_to_tof)
print(stereo_pcd)
print(tof_pcd)

PointCloud with 686890 points.
PointCloud with 244494 points.


In [166]:
# Flip it, otherwise the pointcloud will be upside down
stereo_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
tof_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

PointCloud with 244494 points.

In [218]:
stereo_downpcd = stereo_pcd.voxel_down_sample(voxel_size=0.05)
tof_downpcd = tof_pcd.voxel_down_sample(voxel_size=0.03)
print(stereo_downpcd)
print(tof_downpcd)

PointCloud with 17359 points.
PointCloud with 8583 points.


In [219]:
showPointClouds([stereo_downpcd])

In [220]:
showPointClouds([tof_downpcd])

In [237]:
def processCloud(cloud):
  # Статистически удаляем выбросы
  cloud, ind_f1 = cloud.remove_statistical_outlier(nb_neighbors=20,
                                                      std_ratio=0.01)

  # Удаляем выбросы по окрестности
  cloud, ind_f2 = cloud.remove_radius_outlier(nb_points=15, radius=0.15)

  # outliers = mergeLists(ind_f1, ind_f2)

  # Детектируем плоскость пола и удаляем ее
  plane_model, inliers = cloud.segment_plane(distance_threshold=0.04,
                                          ransac_n=3,
                                          num_iterations=1000)

  cloud = cloud.select_by_index(inliers, invert=True)

  return cloud

res_cloud = processCloud(tof_downpcd)

# showPointClouds([res_cloud])
# display_inlier_outlier(tof_downpcd, outliers)

# Детектируем плоскость портала двери
door_plane, inliers = res_cloud.segment_plane(distance_threshold=0.02,
                                          ransac_n=3,
                                          num_iterations=1000)

[a,b,c,d] = door_plane
axis = np.array([b,a,0])
angle = np.arccos(c)

axis = (axis / np.linalg.norm(axis)) * axis

R = res_cloud.get_rotation_matrix_from_axis_angle(axis)
res_cloud.rotate(R, center=(0,0,0))

res_cloud.translate((0,0,2.4))
print(d)

display_inlier_outlier(res_cloud, inliers)

res_cloud = res_cloud.select_by_index(inliers, invert=True)
print(door_plane)


-0.6703714993541678
Showing outliers (red) and inliers (gray): 


[ 0.04334312  0.9968217  -0.06684214 -0.6703715 ]


In [214]:
[a,b,c,d] = door_plane
print(a,b,c,d)

0.041009278732615596 0.9982867862336103 -0.04173405670672954 -0.5467383320854025


In [217]:

# [a,b,c,d] = [1.0, 1.0, 1.0, 0.0]

xd = np.linspace(-0.5, 0.01, 10)
yd = np.linspace(-0.5, 0.01, 10)
xx, yy = np.meshgrid(xd, yd)
zd = -(a * xx + b * yy + d) / c

fig = go.Figure(data=[
    go.Surface(x=xx, y =yy, z=zd)

])

fig.show()

In [222]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        res_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

max_label = labels.max()
clusters = []
for i in range(max_label + 1):
  cl_cl = o3d.geometry.PointCloud()
  cl_cl.points = o3d.utility.Vector3dVector(np.asarray(res_cloud.points)[labels == i])
  cl_cl.colors = o3d.utility.Vector3dVector(np.asarray(res_cloud.colors)[labels == i])
  clusters.append(cl_cl)

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
res_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
showPointClouds([res_cloud])
# showPointClouds(clusters)

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10


In [223]:
# Рисуем кластеры с bounding box'ами
bb = []
for cl in clusters:
  aabb = cl.get_axis_aligned_bounding_box()
  aabb.color = (1, 0, 0)
  bb.append(aabb)
  # print(bb)
# clusters.extend(bb)
# showPointClouds(bb)
showPointCloudsBB(clusters, bb)

In [ ]:
angle = 
res_cloud.get_rotation_matrix_from_axis_angle()

In [224]:
np.ones((3,1))

array([[1.],
       [1.],
       [1.]])